In [1]:
!pip install pandas groq tiktoken getpass


Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement getpass (from versions: none)
ERROR: No matching distribution found for getpass


In [2]:
import getpass
from groq import Groq
import time

In [4]:
groq_api_key = getpass.getpass(prompt="Insira a tua GROQ API_KEY: ")


Insira a tua GROQ API_KEY: ········


In [5]:
client = Groq(api_key=groq_api_key)
# Modelo LLaMA 4 Scout Instruct
model = "meta-llama/llama-4-scout-17b-16e-instruct"

total_tokens_used = 0

def run_llama_prompt(prompt: str) -> str:
    global total_tokens_used
    completion = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
        max_tokens=512
    )
    usage = getattr(completion, 'usage', None)
    if usage and hasattr(usage, 'total_tokens'):
        total_tokens_used += usage.total_tokens
    return completion.choices[0].message.content.strip()

# Score validation
def validate_score(score: str) -> float:
    try:
        val = float(score)
    except:
        return 0.0
    if val < -5:
        return -5.0
    if val > 10:
        return 10.0
    return val

# Scoring functions
def score_skills(skills: str, job_desc: str) -> float:
    prompt = (
        f"Given the candidate's skills: {skills}\n"
        f"And the job description: {job_desc}\n"
        "Rate on -5 to 10. Respond with just the number."
    )
    return validate_score(run_llama_prompt(prompt))


def score_education(education: str, job_desc: str) -> float:
    prompt = (
        f"Given the candidate's education: {education}\n"
        "Award extra points for high GPA (no penalties for low).\n"
        f"Job desc: {job_desc}\n"
        "Rate on -5 to 10. Respond with just the number."
    )
    return validate_score(run_llama_prompt(prompt))


def score_experience(experience: str, job_desc: str) -> float:
    prompt = (
        f"Given the candidate's experience: {experience}\n"
        f"And the job description: {job_desc}\n"
        "Rate on -5 to 10. Respond with just the number."
    )
    return validate_score(run_llama_prompt(prompt))


def score_summary(summary: str, job_desc: str) -> float:
    prompt = (
        f"Given the candidate's summary: {summary}\n"
        f"And the job description: {job_desc}\n"
        "Rate on -5 to 10. Respond with just the number."
    )
    return validate_score(run_llama_prompt(prompt))



In [11]:
import json, pandas as pd

# Load job descriptions (JobDescription1.json ... JobDescription5.json)
jobs = []
for i in range(1, 6):
    fname = f'JobDescription{i}.json'
    with open(fname, 'r') as f:
        jobs.append(json.load(f))

# Load apenas o CV do candidato 1 (cv_1.json)
with open('cv_1.json', 'r') as f:
    profile = json.load(f)

# 4. Avaliação e agregação de scores
results = []
# Usar apenas o único profile carregado
# profile é um dict JSON do cv_1.json
for job in jobs:
    # Concatenate job text fields
    job_text = (
        job.get('about_the_role', '') + ' ' +
        job.get('key_responsibilities', '') + ' ' +
        job.get('requirements', '')
    )
    # Use job-specific weights
    w = job.get('weights', {"summary":1, "experience":1, "education":1, "skills":1})
    total_w = sum(w.values()) if sum(w.values()) != 0 else 1
    # Compute individual scores
    s_sk = score_skills(profile.get('skills', ''), job_text)
    s_ed = score_education(profile.get('education', ''), job_text)
    s_ex = score_experience(profile.get('experience', ''), job_text)
    s_sm = score_summary(profile.get('summary', ''), job_text)
    # Weighted final score
    final = (
        w.get('skills',0)*s_sk +
        w.get('education',0)*s_ed +
        w.get('experience',0)*s_ex +
        w.get('summary',0)*s_sm
    ) / total_w
    results.append({
        'candidate_name': profile.get('name', ''),
        'job_title': job.get('title', ''),
        'final_score': final
    })

# 5. Exibir resultados. Exibir resultados
results_df = pd.DataFrame(results)
print("Final Scores:")
print(results_df)
print(f"Total tokens used by Groq API: {total_tokens_used}")

# 6. Exportar para CSV, se desejado
results_df.to_csv('profiles_scored.csv', index=False)
print("Results saved to profiles_scored.csv")


Final Scores:
   candidate_name                                  job_title  final_score
0  Carla Nogueira              Directors, Data & AI Strategy         -3.6
1  Carla Nogueira         Senior Consultant, Technology Risk         -1.1
2  Carla Nogueira             Cybersecurity Team Recruitment         -4.5
3  Carla Nogueira                          Financial Analyst          0.6
4  Carla Nogueira  Consultant, Forensic & Integrity Services          1.7
Total tokens used by Groq API: 24238
Results saved to profiles_scored.csv
